# ScoreSight: Complete Data Engineering Pipeline

**Author:** Prathamesh Fuke  
**Project:** EPL Football Analytics & Prediction  
**Branch:** Prathamesh_Fuke  
**Date:** November 13, 2025

## Notebook Overview

This notebook consolidates all data engineering tasks from the original 7 notebooks (01-07) into a single, streamlined pipeline.

### Pipeline Components:
1. **Data Loading & Inspection** - Load raw datasets (Match, Player, League)
2. **Data Cleaning** - Handle missing values, duplicates, standardize columns
3. **Basic Feature Engineering** - Create fundamental features
4. **Advanced Feature Engineering** - Tier 1-3 features (Statistical, Contextual, Interactions)
5. **Data Leakage Validation** - Identify and remove leakage columns
6. **Encoding & Feature Selection** - Prepare final modeling datasets
7. **Visualization** - EDA and feature analysis

### Expected Outputs:
- `../data/cleaned/` - Cleaned datasets
- `../data/features/` - Basic engineered features
- `../data/engineered/` - Advanced v3 features
- `../data/corrected/` - Leakage-corrected datasets
- `../data/final/` - Final modeling-ready data
- `../visualizations/` - EDA charts

---

## Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import skew, kurtosis, percentileofscore
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from pathlib import Path
import warnings
import os

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✓ All libraries imported successfully")

# Create directory structure if not exists
dirs = ['../data/raw', '../data/cleaned', '../data/features', '../data/engineered', 
        '../data/final', '../data/corrected', '../data/encoded', '../visualizations']
for d in dirs:
    os.makedirs(d, exist_ok=True)
print("✓ Directory structure verified")

In [ ]:
print("="*80)
print("PART 1: DATA LOADING & INSPECTION")
print("="*80)

# Load Raw Datasets
# Note: Replace paths with your actual file paths if different
try:
    print("Loading datasets...")
    match_data = pd.read_csv('../datasets/Match Winner.csv')
    player_data = pd.read_excel('../datasets/Goals & Assist.xlsx')
    league_data = pd.read_csv('../datasets/ScoreSight_ML_Season_LeagueWinner_Champion.csv')
    
    print(f"✓ Match Data: {match_data.shape}")
    print(f"✓ Player Data: {player_data.shape}")
    print(f"✓ League Data: {league_data.shape}")
    
    # Save raw copies for reference
    match_data.to_csv('../data/raw/data_raw_match.csv', index=False)
    player_data.to_csv('../data/raw/data_raw_player.csv', index=False)
    league_data.to_csv('../data/raw/data_raw_league.csv', index=False)
    print("✓ Raw datasets saved to ../data/raw/")
    
except FileNotFoundError:
    print("❌ Error: Source datasets not found. Please check paths.")

# Initial Inspection - Match Data
print("\nMatch Data - Missing Values:")
missing = match_data.isnull().sum()
missing = missing[missing > 0]
if not missing.empty:
    print(missing)
else:
    print("No missing values found.")

In [ ]:
print("\n" + "="*80)
print("PART 2: DATA CLEANING")
print("="*80)

# --- Clean Match Data ---
match_clean = match_data.copy()

# Remove duplicates
dups = match_clean.duplicated().sum()
if dups > 0:
    match_clean = match_clean.drop_duplicates()
    print(f"Removed {dups} duplicates from Match Data.")

# Handle Missing Values
numeric_cols = match_clean.select_dtypes(include=[np.number]).columns
categorical_cols = match_clean.select_dtypes(include=['object']).columns

for col in numeric_cols:
    if match_clean[col].isnull().sum() > 0:
        match_clean[col].fillna(match_clean[col].median(), inplace=True)

for col in categorical_cols:
    if match_clean[col].isnull().sum() > 0:
        match_clean[col].fillna(match_clean[col].mode()[0] if not match_clean[col].mode().empty else 'Unknown', inplace=True)

# Standardize Columns
match_clean.columns = match_clean.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')

# --- Clean Player Data ---
player_clean = player_data.copy()
if player_clean.duplicated().sum() > 0:
    player_clean = player_clean.drop_duplicates()

# Fill numeric missing in player data (assumed 0 for stats)
num_cols_player = player_clean.select_dtypes(include=[np.number]).columns
player_clean[num_cols_player] = player_clean[num_cols_player].fillna(0)

player_clean.columns = player_clean.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')

# --- Clean League Data ---
league_clean = league_data.copy()
if league_clean.duplicated().sum() > 0:
    league_clean = league_clean.drop_duplicates()
league_clean.columns = league_clean.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')

# Save Cleaned Data
match_clean.to_csv('../data/cleaned/data_cleaned_match.csv', index=False)
player_clean.to_csv('../data/cleaned/data_cleaned_player.csv', index=False)
league_clean.to_csv('../data/cleaned/data_cleaned_league.csv', index=False)

print(f"✓ Match Data Cleaned: {match_clean.shape}")
print(f"✓ Player Data Cleaned: {player_clean.shape}")
print(f"✓ League Data Cleaned: {league_clean.shape}")

In [ ]:
print("\n" + "="*80)
print("PART 3: BASIC FEATURE ENGINEERING")
print("="*80)

# --- Match Features ---
match_features = match_clean.copy()

# Example Basic Engineering (Customize based on specific column availability)
# Creating simple goal difference if columns exist
if 'htgs' in match_features.columns and 'htgc' in match_features.columns:
    match_features['htgd'] = match_features['htgs'] - match_features['htgc']
    print("Created 'htgd' (Home Team Goal Difference)")

if 'atgs' in match_features.columns and 'atgc' in match_features.columns:
    match_features['atgd'] = match_features['atgs'] - match_features['atgc']
    print("Created 'atgd' (Away Team Goal Difference)")

# --- Player Features ---
player_features = player_clean.copy()
# Basic ratio creation example
if 'goals' in player_features.columns and 'matches_played' in player_features.columns:
    # Avoid division by zero
    player_features['goals_per_game'] = player_features['goals'] / player_features['matches_played'].replace(0, 1)
    print("Created 'goals_per_game'")

# --- League Features ---
league_features = league_clean.copy()

# Save Feature Data
match_features.to_csv('../data/features/data_features_match.csv', index=False)
player_features.to_csv('../data/features/data_features_player.csv', index=False)
league_features.to_csv('../data/features/data_features_league.csv', index=False)

print("✓ Basic feature engineering completed.")

In [ ]:
print("\n" + "="*80)
print("PART 4: ADVANCED FEATURE ENGINEERING (Tier 1-3)")
print("="*80)

# We use the match_features dataframe from the previous step as input
df_match_adv = match_features.copy()

class AdvancedFeatureEngineering:
    def __init__(self, df):
        self.df = df.copy()
        self.feature_descriptions = {}
        # Ensure date column exists and is sorted for rolling calculations
        if 'date' in self.df.columns:
            self.df['date'] = pd.to_datetime(self.df['date'], errors='coerce')
            self.df = self.df.sort_values('date').reset_index(drop=True)

    # --- TIER 1: Statistical & Distributional ---
    def add_tier1_features(self):
        print("  > Generating Tier 1 (Statistical) features...")
        
        # Rolling Window settings
        window = 10
        
        # Percentile Features
        if 'htgs' in self.df.columns and 'atgs' in self.df.columns:
            all_goals = pd.concat([self.df['htgs'], self.df['atgs']])
            self.df['home_goals_percentile'] = self.df['htgs'].apply(lambda x: percentileofscore(all_goals, x, nan_policy='omit'))
            self.df['away_goals_percentile'] = self.df['atgs'].apply(lambda x: percentileofscore(all_goals, x, nan_policy='omit'))
            
        # Rolling Statistics (Skewness, Kurtosis)
        # Note: Using 'hometeam' as grouping key. Adjust if column name differs.
        if 'hometeam' in self.df.columns:
            # Skewness
            self.df['home_goals_skewness'] = self.df.groupby('hometeam')['htgs'].transform(
                lambda x: x.rolling(window=window, min_periods=3).apply(skew, raw=False)
            )
            # Coefficient of Variation
            home_cv = self.df.groupby('hometeam')['htgs'].transform(
                lambda x: x.rolling(window=window, min_periods=3).std() / (x.rolling(window=window, min_periods=3).mean() + 0.1)
            )
            self.df['home_scoring_cv'] = home_cv.fillna(0)
            
            # Quantiles
            for q in [0.25, 0.75]:
                self.df[f'home_goals_q{int(q*100)}'] = self.df.groupby('hometeam')['htgs'].transform(
                    lambda x: x.rolling(window=window, min_periods=3).quantile(q)
                )
            self.df['home_goals_iqr'] = self.df['home_goals_q75'] - self.df['home_goals_q25']

    # --- TIER 2: Market & Context ---
    def add_tier2_features(self):
        print("  > Generating Tier 2 (Contextual) features...")
        
        # Team Quality Proxy (Historical Performance)
        if 'hometeam' in self.df.columns and 'htp' in self.df.columns: # htp = home team points
            team_quality = self.df.groupby('hometeam')['htp'].mean().to_dict()
            quality_vals = list(team_quality.values())
            if quality_vals:
                q_min, q_max = min(quality_vals), max(quality_vals)
                self.df['home_quality_score'] = self.df['hometeam'].map(
                    lambda x: (team_quality.get(x, 0) - q_min) / (q_max - q_min + 1e-5)
                )
        
        # Scheduling (Days Rest)
        if 'date' in self.df.columns:
            home_days = self.df.groupby('hometeam')['date'].transform(lambda x: x.diff().dt.days)
            self.df['home_days_rest'] = home_days.fillna(7)
            self.df['is_midweek'] = self.df['date'].dt.weekday.isin([0, 1, 2, 3]).astype(int)

    # --- TIER 3: Interactions & Non-Linear ---
    def add_tier3_features(self):
        print("  > Generating Tier 3 (Interaction) features...")
        
        # Log Transforms
        if 'htgs' in self.df.columns:
            self.df['home_goals_log'] = np.log1p(self.df['htgs'].fillna(0))
            
        # Efficiency Ratios
        if 'htgs' in self.df.columns and 'htgc' in self.df.columns:
            self.df['home_offensive_efficiency'] = self.df['htgs'] / (self.df['htgs'] + self.df['htgc'] + 1)
            
        # Composite Index (Weighted average of form, goals, defense)
        # Simplified version for stability
        if {'htformpts', 'htgs', 'htgc'}.issubset(self.df.columns):
            scaler = MinMaxScaler()
            # Check if columns have data before scaling
            if not self.df[['htformpts', 'htgs', 'htgc']].isnull().all().any():
                try:
                    comps = np.column_stack([
                        self.df['htformpts'].fillna(0),
                        self.df['htgs'].fillna(0),
                        -self.df['htgc'].fillna(0) # Negative because lower conceded is better
                    ])
                    comps_norm = scaler.fit_transform(comps)
                    weights = np.array([0.4, 0.3, 0.3])
                    self.df['home_strength_index'] = np.dot(comps_norm, weights)
                except Exception as e:
                    print(f"    ! Could not calculate strength index: {e}")

    def execute(self):
        self.add_tier1_features()
        self.add_tier2_features()
        self.add_tier3_features()
        return self.df

# Execute Advanced Engineering
# We check if sufficient columns exist to run this; otherwise we skip to avoid errors in empty demo datasets
required_cols = ['htgs', 'atgs', 'hometeam', 'date']
if set(required_cols).issubset(df_match_adv.columns):
    engineer = AdvancedFeatureEngineering(df_match_adv)
    match_engineered = engineer.execute()
    
    # Save
    match_engineered.to_csv('../data/engineered/data_engineered_match_v3.csv', index=False)
    print(f"✓ Advanced features created. New Shape: {match_engineered.shape}")
else:
    print("! Skipping Advanced Engineering: Required columns (htgs, atgs, hometeam, date) not found.")
    match_engineered = match_features.copy()

In [ ]:
print("\n" + "="*80)
print("PART 5: DATA LEAKAGE VALIDATION & CORRECTION")
print("="*80)

# Define Safe Features (Pre-match available only)
# Matches shouldn't use Full Time Results (FTR) or Goals to predict themselves unless they are lag/rolling features
SAFE_MATCH_FEATURES = [
    'htgs', 'atgs', 'htgc', 'atgc', 'htp', 'atp', 'mw', 
    'htformpts', 'atformpts', 'htstreak', 'atstreak',
    'home_days_rest', 'home_quality_score', 'home_strength_index',
    'home_goals_log', 'home_offensive_efficiency' # Tier 3 features
]

# Define Targets
TARGET_COLS = ['ftr', 'match_result', 'fthg', 'ftag'] # Full Time Result, Goals

def validate_and_clean_leakage(df, dataset_name, safe_list, target_list):
    print(f"\nValidating {dataset_name}...")
    
    df_cols = df.columns.tolist()
    leakage_candidates = []
    
    # Check for columns that look like future info but aren't in safe list or target list
    # (Simple heuristic based on column names)
    for col in df_cols:
        col_lower = col.lower()
        # If it looks like a result but isn't a historical/rolling stat
        if ('result' in col_lower or 'winner' in col_lower or 'points_per_game' in col_lower) \
           and col not in safe_list and col not in target_list:
             # Exclude rolling/lagged features usually denoted by prefixes or suffixes
             if not any(x in col_lower for x in ['last', 'prev', 'rolling', 'mean', 'lag']):
                 leakage_candidates.append(col)
    
    if leakage_candidates:
        print(f"  ! Potential leakage columns detected: {leakage_candidates}")
    else:
        print("  ✓ No obvious leakage columns found based on heuristics.")
        
    # Create Corrected Dataset (Keep Safe + Target)
    # We keep all columns, but explicit "Keep" list ensures we strictly separate X and y later
    # Here we just remove blatant leakage if identified
    
    # Specific Fix for League Data (from NB 06 logic)
    if dataset_name == "League Data" and 'points_per_game' in df.columns:
        print("  ! Removing 'points_per_game' (Derived from target)")
        df = df.drop(columns=['points_per_game'])
        
    return df

# Apply Validation
match_corrected = validate_and_clean_leakage(match_engineered, "Match Data", SAFE_MATCH_FEATURES, TARGET_COLS)
league_corrected = validate_and_clean_leakage(league_features, "League Data", [], ['target_total_points', 'target_league_position'])
player_corrected = validate_and_clean_leakage(player_features, "Player Data", [], ['goals', 'assists'])

# Save Corrected Data
match_corrected.to_csv('../data/corrected/match_prediction_corrected.csv', index=False)
league_corrected.to_csv('../data/corrected/league_winner_corrected.csv', index=False)
player_corrected.to_csv('../data/corrected/top_scorer_corrected.csv', index=False)

print("✓ Datasets validated and saved to ../data/corrected/")

In [ ]:
print("\n" + "="*80)
print("PART 6: ENCODING & FEATURE SELECTION")
print("="*80)

# Process Match Data
match_final = match_corrected.copy()

# 1. Encode Categorical Variables
cat_cols = match_final.select_dtypes(include=['object']).columns.tolist()
# Exclude Date
if 'date' in cat_cols: cat_cols.remove('date')

label_encoders = {}
for col in cat_cols:
    # Only encode if low cardinality, otherwise likely identifiers
    if match_final[col].nunique() < 50:
        le = LabelEncoder()
        match_final[f'{col}_encoded'] = le.fit_transform(match_final[col].astype(str))
        label_encoders[col] = le
        print(f"  Encoded {col}")

# 2. Select Numeric Features for Modeling
# We exclude the raw categorical columns and identifiers
numeric_final = match_final.select_dtypes(include=[np.number]).copy()

# 3. Drop Target from Features (if it exists in numeric form) but keep for y
if 'fthg' in numeric_final.columns:
    # Assuming fthg (Full Time Home Goals) is a target, we shouldn't use it as input
    # But we might need it for correlation analysis in Part 7
    pass 

# Save Final Modeling Datasets
match_final.to_csv('../data/final/data_final_match_prediction.csv', index=False)
# (Repeat for player/league if needed, usually match is the primary modeling target)

print(f"✓ Final Match Dataset Ready: {match_final.shape}")

In [ ]:
print("\n" + "="*80)
print("PART 7: VISUALIZATION")
print("="*80)

# 1. Correlation Heatmap
numeric_cols = match_final.select_dtypes(include=[np.number]).columns
# Filter to interesting columns to avoid clutter
interesting_cols = [c for c in numeric_cols if 'encoded' in c or c in SAFE_MATCH_FEATURES or c in ['fthg', 'ftag']]

if len(interesting_cols) > 1:
    plt.figure(figsize=(12, 10))
    corr_matrix = match_final[interesting_cols].corr()
    sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0, linewidths=0.5)
    plt.title('Match Data - Feature Correlation Heatmap', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('../visualizations/viz_match_correlation.png')
    print("✓ Saved Correlation Heatmap")
    plt.show()

# 2. Feature Distributions (Tier 1-3 Features)
viz_features = ['home_goals_percentile', 'home_scoring_cv', 'home_strength_index', 'home_quality_score']
valid_viz = [f for f in viz_features if f in match_final.columns]

if valid_viz:
    fig, axes = plt.subplots(1, len(valid_viz), figsize=(16, 5))
    if len(valid_viz) == 1: axes = [axes]
    
    for idx, feature in enumerate(valid_viz):
        data = match_final[feature].dropna()
        axes[idx].hist(data, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
        axes[idx].set_title(feature, fontweight='bold')
        axes[idx].set_xlabel('Value')
        axes[idx].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.savefig('../visualizations/viz_feature_distributions.png')
    print("✓ Saved Feature Distributions")
    plt.show()

# 3. Summary Dashboard
print("\nData Pipeline Summary:")
print(f"  Total Records Processed: {match_final.shape[0]}")
print(f"  Total Features Generated: {match_final.shape[1]}")
print(f"  Final data saved to: ../data/final/data_final_match_prediction.csv")